# Excel versus Python - High Level

You are familiar with Excel so we'll start with a quick comparison between spreadsheets and Python tools.   You'll see that they're very much alike - at least at first glance.

Let's suppose that you have a comma separated value (.csv) file that has information about a few sports, how long matches take to play, and how many fans typically attend an event.

Let's look at that file in Excel and pandas (Python's "spreadsheet analog" tool).

In [10]:
#In Python, we let the system know what tools we'll need
import pandas as pd
from numpy import NaN
import sys
import os

#This is a utility for displaying content side-by-side
from custom_utils.display_wide import  display_wide

#Some file names
csv_file_name = os.path.abspath(os.path.join(os.getcwd(), "support_files", "sports.csv"))
ex_img_fn = os.path.join("support_files", "excel_image.JPG")
csv_img_fn = os.path.join("support_files", "csv_image_.PNG")

In [11]:
#Here's a way to read a file with pandas                         
sports = pd.read_csv(csv_file_name)

display_wide([csv_img_fn, ex_img_fn, sports, ], ["CSV", "Excel", "Pandas"])

As you can see, the information contained in the .csv file was ingested easily by both Excel and Python. In Excel you use a "File .. Open" dialog.  In in pandas you execute a command.

If you look closely, you'll see that the row and column headers are a little different.   We have lots of control over these.  

Don't worry about the details at this point, but let's let's make an Excel look-alike and a version that works best in Pandas:

In [12]:
#Read in the data again and make it look like Excel
sports_excel = pd.read_csv(csv_file_name, header=None) 

#Set the row and column names
sports_excel.columns = ["A", "B", "C"]
sports_excel.index = pd.Index([1,2,3,4])

#Read in the original data with a column as the row index
sports = pd.read_csv(csv_file_name, index_col='sport')

display_wide([sports_excel, sports, ], ['Excel Style', 'Pandas Style'], 3)

Let's take a moment and compare the "Excel version" and the "pandas version" carefully.

On the left, you'll see an Excel knock-off; on the right you'll a more accessible panda object.  A couple of things to notice:

- The Excel version has columns "A", "B", "C" and so on.  This is OK, but what's intuitive about "C"?  Nothing really.  The row numbers don't mean much either.   So a reference to "Sheet1!C3" is nearly-meaningless unless you're staring at the spreadsheet.    


- The same issue of intuitiveness applies to the row designations.  In pandas we can use the contents of one of the columns, dates, or anything else we want. If we want to find how many fans attend an average wresting event, we could simply locate them like this:

<blockquote><b>sports.loc['wrestling', 'fans']</b>

... which the newest team member will instantly understand.
</blockquote>

- Finally, you'll notice that the row and column lables are relegated to index values.  They're not part of the data.  This is really clean because the data can be all numbers - so operations like addition and subtraction will always work.  In fact, in pandas every value in a column is the same data type (all integers, all floating points, etc.)  That takes a load off under-the-hood operations and makes computations extremely efficient.

In [13]:
attendees = sports.loc['wrestling', 'fans']
print("\nWrestling has {} average attendees.".format(attendees))


Wrestling has 300 average attendees.


## Add and Delete Rows/Columns

In Excel, you use the GUI to get around.   You don't really have to add rows or columns since the spreadsheet gets as big as you need it to be automatically.  To add content you simply type it into an empty cell.

In Pandas the data-containing object can automatically get as big as needs to be, too.  You can force it to grow by using code to add new content.   

I'll show you some simple ways to add and remove content here.  These are <u>not</u> the most efficient but they're sensible first steps.

Let's tackle <u>columns</u> first.   To add a column, you simply announce its name and what it holds.  Let's add the number of players required. 


In [14]:
#We can add a new column using a list-like object
sports_players = sports.copy()
sports_players['players'] = [18, 2, 2]

#We might just add an empty column with the special object nan
sports_empty = sports.copy()
sports_empty['empty'] = NaN

#To get rid of a column we can use the keyword 'del' (short for 'delete')
sports_empty_removed = sports_empty.copy()
del sports_empty_removed['empty']
del sports_empty_removed['duration']

display_wide([sports_players, sports_empty, sports_empty_removed],
            ['sports_players', 'sports_empty', 'sports_empty_removed']
            )

You'll note that we've introduced the object "<u>NaN</u>" into the mix.  This is a special placeholder in pandas - sort of like #NULL! in Excel.  It politely drops out of many calculations.  If you use mean() you'll the average of available values.  If you try simple addition, multiplication, etc., the result is NaN. You'll note that it's treated as a floating point number and forced both the 'duration' and 'fans' columns to show up with decimal points.

<u>Rows</u> work just about the same way.   You can introduce a new row's content by providing a new index value and data for the columns.   To get rid of a new row, you can use the drop() method.  

Here, we're adding two new sports 'soccer' and 'basket-weaving' essentially be asserting they exist and providing the duration and number of fans for ezch.

Next, we're removing 'soccer' from a copy of the augmented data object we just created.

In [16]:
#Start with a fresh copy
sports_new_rows = sports.copy()

#Here, we're adding two new sports to our DataFrame
sports_new_rows.loc['soccer'] = [NaN, NaN]
sports_new_rows.loc['basket-weaving'] = [600, 0]

#This uses the drop() command go get rid of soccer
sports_removed_rows = sports_new_rows.copy().drop('soccer')

display_wide([sports_new_rows, sports_removed_rows], spacing = 3)

## "Spreadsheet Formulas" in pandas

In Excel, you type a formula in every cell you want calculated values.   For instance if you wanted to add a values in a couple of adjacent columns you might go "=Sheet1!A1 + Sheet1!B1", then copy the formula down the column or across the rows.   

Pandas lets you do created calcuated values, and with some significant upgrades:

- Operations can occur on an entire column without the need to think about every row.  As a result you can't screw things up by failing to copy an entire row or column as you propogate equations.

- You can dynamically name the new columns as you create them.  You don't need to provide column headings, range names, etc. as a separate step.

Here are a few examples using built-in operators which will be familiar to any Excel user;

In [ ]:
#Start with a fresh copy
sports_ops = sports.copy()

#Arithmatic ops like +, -, *, /, % all work as expected
sports_ops['fans_x_2'] = sports_ops['fans'] * 2

#Tests like >, <, >=, <=, != all return Boolean values
sports_ops['lots_of_fans'] = sports_ops['fans'] > 200

#Bring in the name of an Excel spreadsheet screenshot
formulas_img_fn = os.path.join("support_files", "sports_formulas.jpg")

display_wide([formulas_img_fn, sports_ops ],["Excel", "Pandas"])


You'll note that you didn't have to tell pandas anything about the new column of data except instructions about how to create the content.   The data type is chosen automatically.

In this sense pandas works just like Excel.  Much of the housekeeping around memory allocation, data type selection, etc. is handled for you.  You'll get error messages if you try to do math on strings or string operations on numbers.

In [ ]:
display_wide([sports_ops, sports_ops.dtypes.to_frame()], ['Data', "Data Types"])

## Now how the %#@@ did I get THAT number?

Have you ever gone back to a giant, complicated spreadsheet you worked on months ago to figure out how some value was calculated?   Or where some mistake could have happened?    

Good.  That makes you precisely human.

As you review this section, please think about the simplicity and transparancy you're already able to gain by using Python - and how it will help you analyze your previous work.

So far you've been able to:

- Eliminate the possibility of making a mechanical mistake.  You can't screw up a 'drag and drop' operation when propogating an equation, say.  You don't have to worry about things like getting relative references right, pasting the right number of columns, etc.

- Create a "recipe" for analyzing columns of data.  When you go to audit the process, you're looking at a few equations instead of thousands of individual data cells.

- Make calculations intuitive and based in clear English - no more guesswork because you've upgraded:
             "Sheet1!B764"  -- > can become --> "dog.loc['Fido', 'favorite_treat']"

Great progress so far!